# Overview

LLM應用中幾個較為有的應用當數複雜的問答 (Q&A) 聊天機器人。這些應用程式可以回答有關特定來源資訊的問題。這些應用程式使用一種稱為檢索增強生成（RAG）的技術。

# 什麼是 RAG

RAG 是一種利用額外資料來增強 LLM 知識的技術。

LLM可以用來回應廣泛的主題，但他們的知識僅限於他們接受訓練的的數據。如果您想要建立能夠推理私有資料或模型截止日期之後引入的資料的 AI 應用程序，您需要使用模型所需的特定資訊來增強模型的知識。引入適當的資訊並將其插入模型提示的過程稱為檢索增強生成 (RAG)。

LangChain 有許多元件旨在幫助建立問答應用程式以及更廣泛的 RAG 應用程式。

注意：這裡我們將重點放在非結構化資料的問答。我們在其他地方介紹的兩個 RAG 用例是：

1.  [SQL資料問答](https://python.langchain.com/docs/use_cases/sql/)
2.  [程式碼問答（例如 Python）](https://python.langchain.com/docs/use_cases/code_understanding/)

# RAG Architecture

典型的 RAG 應用程式有兩個主要組件：

索引：用於從來源取得資料並為其建立索引的管道。 這通常發生在離線狀態。

檢索和生成：實際的 RAG 鏈，它在運行時接受用戶查詢並從索引中檢索相關數據，然後將其傳遞給模型。

從原始資料到答案的最常見的完整序列如下所示：

索引

1.   載入：首先我們需要載入資料。 這是透過[DocumentLoaders](https://python.langchain.com/docs/use_cases/question_answering/#:~:text=is%20done%20with-,DocumentLoaders,-.) 完成的。
2.   分割：[文字分割器](https://python.langchain.com/docs/use_cases/question_answering/#:~:text=Split%3A-,Text%20splitters,-break%20large%20Documents)將大文檔分成更小的區塊。 這對於索引資料和將其傳遞到模型都很有用，因為大塊更難搜尋並且不適合模型的有限上下文視窗。
3.    儲存：我們需要某個地方來儲存和索引我們的分割，以便以後可以搜尋它們。 這通常是使用 [VectorStore](https://python.langchain.com/docs/modules/data_connection/vectorstores/) 和 [Embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/) 模型來完成的。




![範例1](https://raw.githubusercontent.com/markl-a/LLM-agent-Demo/main/1.LangchainDemo/1.langchain%E5%AE%98%E7%B6%B2%E4%BD%BF%E7%94%A8%E7%AF%84%E4%BE%8B%EF%BC%9ARAG%E5%95%8F%E7%AD%94/rag_indexing-1.png)

檢索和生成


1.   檢索：給定使用者輸入，使用[檢索器](https://python.langchain.com/docs/modules/data_connection/retrievers/)從儲存中檢索相關分割。
2.   產生：[ChatModel](https://python.langchain.com/docs/modules/model_io/chat/) / [LLM](https://python.langchain.com/docs/modules/model_io/llms/) 使用包含問題和檢索到的資料的提示產生答案





![範例2](https://raw.githubusercontent.com/markl-a/LLM-agent-Demo/main/1.LangchainDemo/1.langchain官網使用範例：RAG問答/rag_retrieval_generation-2.png)

# 目錄



*   快速入門：我們建議從這裡開始。 以下許多指南假設您完全理解快速入門中所示的體系結構。
*   傳回來源：如何傳回特定生成方法中使用的來源文件。
*   串流傳輸：如何串流最終答案以及中間步驟。
*   新增聊天記錄：如何將聊天記錄新增至問答應用程式。
*   按使用者檢索：每個使用者都有自己的私有資料時如何進行檢索。
*   使用代理：如何使用代理進行問答。
*   使用本地模型：如何使用本地模型進行問答。








In [2]:
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.1/289.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os

# for LLM model
os.environ["OPENAI_API_KEY"] = "填入你的OpenAI API KEY"

# for langSmith (可以不使用)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "填入你的LangSmith API KEY(非必要)"

import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 讀取資料，轉為向量儲存
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
print('完成向量化')


# 進行檢索與生成
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print("完成建立RAG")

result = rag_chain.invoke("What is Task Decomposition? 請用繁體中文回答。")
print(result)

完成向量化
完成建立RAG
任務分解是將問題分解為多個思考步驟，每個步驟生成多個想法，創建樹狀結構。可以通過簡單提示、任務特定指令或人類輸入來進行任務分解。這有助於將大型任務轉換為多個可管理的任務。


In [7]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is. """
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """你是問答任務的助手。 \
使用以下檢索到的上下文來回答問題。 \
如果你不知道答案，就說你不知道。 \
最多使用三個句子並保持答案簡潔\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [10]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Task Decomposition? 請用繁體中文回答。"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "What are common ways of doing it? 請用繁體中文回答。"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

進行任務分解的常見方式包括使用簡單提示的LLM、使用任務特定指令，以及人類的輸入。
